In [1]:
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import datetime
import os
import json
import time

In [83]:
os.getcwd()

In [16]:
url_rebtel = 'https://www.trustpilot.com/review/www.rebtel.com'
# url = 'https://www.trustpilot.com/review/www.rebtel.com?languages=all'
url_rebtel_page = url_rebtel + '?page='

In [89]:
with urllib.request.urlopen(url_rebtel) as url:
    homepage = url.read()

In [90]:
soup = BeautifulSoup(homepage, 'html.parser')
# print(soup.prettify())

In [91]:
# list(soup.children)
# soup.head
print (soup.prettify()[0:200])

<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   Rebtel Reviews | Customer Service Reviews of Rebtel | www.rebtel.com
  </title>
  <meta charset="utf-8"/>
  <script type="text/javascript">
   win


In [92]:
reviews = soup.find_all('div', class_='review-stack')
container = soup.find_all('div', class_='reviews-container')
len(reviews)

20

In [21]:
pagination = soup.find_all('a', class_='pagination-page')
page_num = [page['data-page-number'] for page in pagination]
page_num = [int(page) for page in page_num if page != 'next-page']
page_max = max(page_num)
page_max

112

In [77]:
# Total number of reviews
# reviews_num = soup.find('span', class_='ratingCount').get_text()
# reviews_num = int(reviews_num.replace(',',''))
# reviews_num
# Total number of pages
# pages_num = math.ceil(num_reviews / 20)
# pages_num

In [143]:
data = []  
for page in range(1, page_max + 1):
    
    if (page == 1):
        url_complete = url_rebtel
    else:
        url_complete = url_rebtel_page + str(page)
    
    with urllib.request.urlopen(url_complete) as url:
            page_content = url.read()
        
    soup = BeautifulSoup(page_content, 'html.parser')
    reviews = soup.find_all('div', class_='review-stack')
    
    for i in range(0, len(reviews)):       
        user_id = reviews[i].find('a', class_='user-review-name-link')['href'].strip()
        user_id = user_id.replace('/users/', '')
        
        user_name = reviews[i].find('a', class_='user-review-name-link').get_text().strip()
        
        review_title = reviews[i].find('a', class_='review-title-link')
        review_title = review_title.get_text().strip()
        
        review_text = reviews[i].find('div', class_='review-body').get_text().strip()
        
        review_answer = reviews[i].find('div', class_='comment')
        if (review_answer is not None):
            review_answer = review_answer.get_text().strip()
        else:
            review_answer = None
        
        review_stars = int(reviews[i].find('div', class_='social-share-network social-share-network--twitter')['data-status'].split()[-5])
        
        review_date = reviews[i].find('time', class_='ndate')['datetime'].strip()
        review_date = review_date.replace('.000+00:00', '')
        review_date = review_date.replace('T', ' ')

        review_verify_text = reviews[i].find('div', class_='review-verified')
        if (review_verify_text is not None):
            review_verify = 'Verified order' in review_verify_text.get_text()
        else:
            review_verify = False        
        
        data.append({  
            'user_id': user_id,
            'user': user_name,
            'title': review_title,
            'text': review_text,
            'answer': review_answer,
            'stars': review_stars,
            'date': review_date,
            'verify': review_verify
        })
    

In [137]:
today = time.strftime('%Y%m%d')
file_name = 'data/rawdata_' + today + '.json'

In [152]:
with open(file_name, 'w') as outfile:  
    json.dump(data, outfile, indent = 2)